## Coastal Upwelling/Downwelling Dynamics - What happens when wind blows along the coast?

<img src="EA_WAOR_2018_labels.png" width="400" align="right">

Using real-world observations we will have a look. Using python jupyter notebook you will use observations from the Ocean Observing Institute (OOI) Coastal Endurance Array offshore of Washington. The array contains three mooring locations located at shallow, mid, and deep waters (see image on right)

The data is availabler through OOI's data portal, https://oceanobservatories.org/array/coastal-endurance/. Here we will utilize the OOI API interface to download and analyze data inside this python notebook

To get a sense for what these instruments look like, chechk out this sped-up video of the Oregon endurance array installation, https://www.youtube.com/watch?time_continue=13&v=snw6XzyV3dQ&feature=emb_logo



<img src="Coastal-Endurance-Array.jpg" width="400" align="right">

In [1]:
# Notebook Setup
import requests
import os
import re
import xarray as xr
import dask
import matplotlib.pyplot as plt
import numpy as np

# Setup default plot styles
import seaborn as sns
sns.set()

In [2]:
API_USERNAME = 'OOIAPI-RHP6L5EUN3FOVV'
API_TOKEN = 'TEMP-TOKEN-HXJ17KWY5WRYZG'

In [3]:
# Washing nearshore CTD profiler, url https://ooinet.oceanobservatories.org/data_access/?search=CE06ISSP-SP001-09-CTDPFJ000#CE06ISSP-SP001-09-CTDPFJ000
site = 'CE06ISSP'
node = 'SP001'
instrument = '09-CTDPFJ000'
method = 'telemetered'
stream = 'ctdpf_j_cspp_instrument'


# Instrument Information (Example)
#site = 'CP04OSSM'
#node = 'SBD11'
#instrument = '06-METBKA000'
#method = 'telemetered'
#stream = 'metbk_a_dcl_instrument'

api_base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv'

# Create the request URL
data_request_url ='/'.join((api_base_url,site,node,instrument,method,stream))

In [4]:
# All of the following are optional, but you should specify a date range
params = {
  'beginDT':'2016-01-01T00:00:00.000Z',
  'endDT':'2017-01-01T00:00:00.000Z',
  'format':'application/netcdf',
  'include_provenance':'true',
  'include_annotations':'true'
}

In [5]:
# Run only once
r = requests.get(data_request_url, params=params, auth=(API_USERNAME, API_TOKEN))
data = r.json()


In [6]:
#data
print(data['allURLs'][0])

https://opendap.oceanobservatories.org/thredds/catalog/ooi/sean.crosby@wwu.edu/20200304T230110062Z-CE06ISSP-SP001-09-CTDPFJ000-telemetered-ctdpf_j_cspp_instrument/catalog.html


In [7]:
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/sean.crosby@wwu.edu/20200302T004136807Z-CE06ISSP-SP001-09-CTDPFJ000-telemetered-ctdpf_j_cspp_instrument/catalog.html'
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i) for i in x]
datasets

['https://opendap.oceanobservatories.org/thredds/dodsC/ooi/sean.crosby@wwu.edu/20200302T004136807Z-CE06ISSP-SP001-09-CTDPFJ000-telemetered-ctdpf_j_cspp_instrument/deployment0005_CE06ISSP-SP001-09-CTDPFJ000-telemetered-ctdpf_j_cspp_instrument_20160727T201022.680000-20160818T201941.685000.nc',
 'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/sean.crosby@wwu.edu/20200302T004136807Z-CE06ISSP-SP001-09-CTDPFJ000-telemetered-ctdpf_j_cspp_instrument/deployment0004_CE06ISSP-SP001-09-CTDPFJ000-telemetered-ctdpf_j_cspp_instrument_20160513T185525.292000-20160518T081645.689000.nc']

In [12]:
selected_datasets = datasets[0]
selected_datasets

'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/sean.crosby@wwu.edu/20200302T004136807Z-CE06ISSP-SP001-09-CTDPFJ000-telemetered-ctdpf_j_cspp_instrument/deployment0005_CE06ISSP-SP001-09-CTDPFJ000-telemetered-ctdpf_j_cspp_instrument_20160727T201022.680000-20160818T201941.685000.nc'

In [9]:
ds = xr.open_dataset(selected_datasets)



OSError: [Errno -68] NetCDF: I/O failure: b'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/sean.crosby@wwu.edu/20200302T004136807Z-CE06ISSP-SP001-09-CTDPFJ000-telemetered-ctdpf_j_cspp_instrument/deployment0005_CE06ISSP-SP001-09-CTDPFJ000-telemetered-ctdpf_j_cspp_instrument_20160727T201022.680000-20160818T201941.685000.nc'

ds = ds.swap_dims({'obs': 'time'}) # Swap the primary dimension
ds = ds.chunk({'time': 100}) # Used for optimization
ds = ds.sortby('time') # Data from different deployments can overlap so we want to sort all data by time stamp.
ds

<img src="upwelling_ca.jpg" width="400" align="right">
Upwelling can really have an impact, take a look at the sattelite image on the right. Wow it's so cold along the coast of California. Offshore waters are 14-15 degrees Celcius (59-deg Fahrenheit) while along the coast it's 10-degrees Celcius (50-deg Fahrenheit).


Even in Southern California, known for it's warm summers, water temperatures can plummet when winds drive significant upwelling. For example, during the summer of 2020 water temperatures in San Diego dropped below 60-deg,
https://www.sandiegouniontribune.com/weather/story/2020-07-31/san-diegos-ocean-temperatures-turn-cold-as-winter-at-height-of-summer. You'll need a wetsuit at that temperature!